In [1]:
import pandas as pd
import numpy as np
import os
from env import host, user, password

import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats

from sklearn.metrics import mean_squared_error, explained_variance_score
from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_regression, RFE

pd.options.display.float_format = '{:20,.2f}'.format

In [2]:
import wrangle_zillow as wr

In [3]:
df = wr.get_zillow_data()
df.head()

,parcelid,id,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,id.1,logerror,transactiondate
0,14297519,1727539,NaN,NaN,NaN,3.50,4.00,NaN,NaN,3.50,...,"1,023,282.00","2,016.00","537,569.00","11,013.72",NaN,NaN,"60,590,630,072,012.00",0,0.03,2017-01-01
1,17052889,1387261,NaN,NaN,NaN,1.00,2.00,NaN,NaN,1.00,...,"464,000.00","2,016.00","376,000.00","5,672.48",NaN,NaN,"61,110,010,023,006.00",1,0.06,2017-01-01
2,14186244,11677,NaN,NaN,NaN,2.00,3.00,NaN,NaN,2.00,...,"564,778.00","2,016.00","479,489.00","6,488.30",NaN,NaN,"60,590,218,022,012.00",2,0.01,2017-01-01
3,12177905,2288172,NaN,NaN,NaN,3.00,4.00,NaN,8.00,3.00,...,"145,143.00","2,016.00","36,225.00","1,777.51",NaN,NaN,"60,373,001,001,006.00",3,-0.10,2017-01-01
4,10887214,1970746,1.00,NaN,NaN,3.00,3.00,NaN,8.00,3.00,...,"119,407.00","2,016.00","45,726.00","1,533.89",NaN,NaN,"60,371,236,012,000.00",4,0.01,2017-01-01


In [4]:
pd.set_option("display.max_columns", None)

In [5]:
df.head()

,parcelid,id,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,hashottuborspa,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid10,pooltypeid2,pooltypeid7,propertycountylandusecode,propertylandusetypeid,propertyzoningdesc,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,storytypeid,threequarterbathnbr,typeconstructiontypeid,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,id.1,logerror,transactiondate
0,14297519,1727539,NaN,NaN,NaN,3.50,4.00,NaN,NaN,3.50,NaN,NaN,"3,100.00","3,100.00",NaN,NaN,NaN,NaN,"6,059.00",NaN,3.00,2.00,633.00,NaN,NaN,"33,634,931.00","-117,869,207.00","4,506.00",NaN,NaN,NaN,NaN,NaN,122,261.00,NaN,"60,590,630.07","53,571.00","1,286.00",NaN,"96,978.00",0.00,NaN,1.00,NaN,NaN,NaN,NaN,"1,998.00",NaN,NaN,"485,713.00","1,023,282.00","2,016.00","537,569.00","11,013.72",NaN,NaN,"60,590,630,072,012.00",0,0.03,2017-01-01
1,17052889,1387261,NaN,NaN,NaN,1.00,2.00,NaN,NaN,1.00,NaN,"1,465.00","1,465.00","1,465.00",NaN,NaN,"1,465.00",NaN,"6,111.00",1.00,1.00,1.00,0.00,NaN,NaN,"34,449,266.00","-119,281,531.00","12,647.00",NaN,NaN,NaN,NaN,NaN,1110,261.00,NaN,"61,110,010.02","13,091.00","2,061.00",NaN,"97,099.00",5.00,NaN,NaN,NaN,NaN,NaN,NaN,"1,967.00",1.00,NaN,"88,000.00","464,000.00","2,016.00","376,000.00","5,672.48",NaN,NaN,"61,110,010,023,006.00",1,0.06,2017-01-01
2,14186244,11677,NaN,NaN,NaN,2.00,3.00,NaN,NaN,2.00,NaN,NaN,"1,243.00","1,243.00",NaN,NaN,NaN,NaN,"6,059.00",NaN,2.00,2.00,440.00,NaN,NaN,"33,886,168.00","-117,823,170.00","8,432.00",1.00,NaN,NaN,NaN,1.00,122,261.00,NaN,"60,590,218.02","21,412.00","1,286.00",NaN,"97,078.00",6.00,NaN,NaN,NaN,NaN,NaN,NaN,"1,962.00",1.00,NaN,"85,289.00","564,778.00","2,016.00","479,489.00","6,488.30",NaN,NaN,"60,590,218,022,012.00",2,0.01,2017-01-01
3,12177905,2288172,NaN,NaN,NaN,3.00,4.00,NaN,8.00,3.00,NaN,NaN,"2,376.00","2,376.00",NaN,NaN,NaN,NaN,"6,037.00",NaN,3.00,NaN,NaN,NaN,2.00,"34,245,180.00","-118,240,722.00","13,038.00",1.00,NaN,NaN,NaN,1.00,0101,261.00,LCR110000*,"60,373,001.00","396,551.00","3,101.00",NaN,"96,330.00",0.00,NaN,NaN,NaN,1.00,NaN,NaN,"1,970.00",NaN,NaN,"108,918.00","145,143.00","2,016.00","36,225.00","1,777.51",NaN,NaN,"60,373,001,001,006.00",3,-0.10,2017-01-01
4,10887214,1970746,1.00,NaN,NaN,3.00,3.00,NaN,8.00,3.00,NaN,NaN,"1,312.00","1,312.00",NaN,NaN,NaN,NaN,"6,037.00",NaN,3.00,NaN,NaN,NaN,2.00,"34,185,120.00","-118,414,640.00","278,581.00",1.00,NaN,NaN,NaN,1.00,010C,266.00,LAR3,"60,371,236.01","12,447.00","3,101.00","268,548.00","96,451.00",0.00,NaN,NaN,NaN,1.00,NaN,NaN,"1,964.00",NaN,NaN,"73,681.00","119,407.00","2,016.00","45,726.00","1,533.89",NaN,NaN,"60,371,236,012,000.00",4,0.01,2017-01-01


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 72241 entries, 0 to 72240
Data columns (total 62 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   parcelid                      72241 non-null  int64  
 1   id                            72241 non-null  int64  
 2   airconditioningtypeid         23079 non-null  float64
 3   architecturalstyletypeid      207 non-null    float64
 4   basementsqft                  50 non-null     float64
 5   bathroomcnt                   72241 non-null  float64
 6   bedroomcnt                    72241 non-null  float64
 7   buildingclasstypeid           0 non-null      float64
 8   buildingqualitytypeid         45080 non-null  float64
 9   calculatedbathnbr             71987 non-null  float64
 10  decktypeid                    612 non-null    float64
 11  finishedfloor1squarefeet      6032 non-null   float64
 12  calculatedfinishedsquarefeet  72055 non-null  float64
 13  f

In [7]:
def remove_columns(df, cols_to_remove):  
    df = df.drop(columns=cols_to_remove)
    return df

In [8]:
def handle_missing_values(df, prop_required_column = .5, prop_required_row = .75):
    threshold = int(round(prop_required_column*len(df.index),0))
    df.dropna(axis=1, thresh=threshold, inplace=True)
    threshold = int(round(prop_required_row*len(df.columns),0))
    df.dropna(axis=0, thresh=threshold, inplace=True)
    return df

In [9]:
def data_prep(df, cols_to_remove=[], prop_required_column=.5, prop_required_row=.75):
    df = remove_columns(df, cols_to_remove)
    df = handle_missing_values(df, prop_required_column, prop_required_row)
    return df

In [10]:
# columns removed for having less than 50% of the values
df = data_prep(df, cols_to_remove=['airconditioningtypeid','architecturalstyletypeid','basementsqft','buildingclasstypeid','decktypeid','finishedfloor1squarefeet','finishedsquarefeet13','finishedsquarefeet15','finishedsquarefeet50','finishedsquarefeet6','fireplacecnt','garagecarcnt','garagetotalsqft','hashottuborspa','poolcnt','poolsizesum','pooltypeid10','pooltypeid2','pooltypeid7','regionidneighborhood','storytypeid','threequarterbathnbr','typeconstructiontypeid','yardbuildingsqft17','yardbuildingsqft26','numberofstories','fireplaceflag','taxdelinquencyflag','taxdelinquencyyear'], prop_required_column=.5, prop_required_row=.75)
df.head()

,parcelid,id,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,fips,fullbathcnt,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,propertycountylandusecode,propertylandusetypeid,propertyzoningdesc,rawcensustractandblock,regionidcity,regionidcounty,regionidzip,roomcnt,unitcnt,yearbuilt,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock,id.1,logerror,transactiondate
0,14297519,1727539,3.50,4.00,NaN,3.50,"3,100.00","3,100.00","6,059.00",3.00,NaN,"33,634,931.00","-117,869,207.00","4,506.00",122,261.00,NaN,"60,590,630.07","53,571.00","1,286.00","96,978.00",0.00,NaN,"1,998.00","485,713.00","1,023,282.00","2,016.00","537,569.00","11,013.72","60,590,630,072,012.00",0,0.03,2017-01-01
1,17052889,1387261,1.00,2.00,NaN,1.00,"1,465.00","1,465.00","6,111.00",1.00,NaN,"34,449,266.00","-119,281,531.00","12,647.00",1110,261.00,NaN,"61,110,010.02","13,091.00","2,061.00","97,099.00",5.00,NaN,"1,967.00","88,000.00","464,000.00","2,016.00","376,000.00","5,672.48","61,110,010,023,006.00",1,0.06,2017-01-01
2,14186244,11677,2.00,3.00,NaN,2.00,"1,243.00","1,243.00","6,059.00",2.00,NaN,"33,886,168.00","-117,823,170.00","8,432.00",122,261.00,NaN,"60,590,218.02","21,412.00","1,286.00","97,078.00",6.00,NaN,"1,962.00","85,289.00","564,778.00","2,016.00","479,489.00","6,488.30","60,590,218,022,012.00",2,0.01,2017-01-01
3,12177905,2288172,3.00,4.00,8.00,3.00,"2,376.00","2,376.00","6,037.00",3.00,2.00,"34,245,180.00","-118,240,722.00","13,038.00",0101,261.00,LCR110000*,"60,373,001.00","396,551.00","3,101.00","96,330.00",0.00,1.00,"1,970.00","108,918.00","145,143.00","2,016.00","36,225.00","1,777.51","60,373,001,001,006.00",3,-0.10,2017-01-01
4,10887214,1970746,3.00,3.00,8.00,3.00,"1,312.00","1,312.00","6,037.00",3.00,2.00,"34,185,120.00","-118,414,640.00","278,581.00",010C,266.00,LAR3,"60,371,236.01","12,447.00","3,101.00","96,451.00",0.00,1.00,"1,964.00","73,681.00","119,407.00","2,016.00","45,726.00","1,533.89","60,371,236,012,000.00",4,0.01,2017-01-01


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 72097 entries, 0 to 72240
Data columns (total 33 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   parcelid                      72097 non-null  int64  
 1   id                            72097 non-null  int64  
 2   bathroomcnt                   72097 non-null  float64
 3   bedroomcnt                    72097 non-null  float64
 4   buildingqualitytypeid         45080 non-null  float64
 5   calculatedbathnbr             71987 non-null  float64
 6   calculatedfinishedsquarefeet  72055 non-null  float64
 7   finishedsquarefeet12          71875 non-null  float64
 8   fips                          72097 non-null  float64
 9   fullbathcnt                   71987 non-null  float64
 10  heatingorsystemtypeid         46697 non-null  float64
 11  latitude                      72097 non-null  float64
 12  longitude                     72097 non-null  float64
 13  l

In [12]:
df = df.drop(df.columns[1], axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 72097 entries, 0 to 72240
Data columns (total 32 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   parcelid                      72097 non-null  int64  
 1   bathroomcnt                   72097 non-null  float64
 2   bedroomcnt                    72097 non-null  float64
 3   buildingqualitytypeid         45080 non-null  float64
 4   calculatedbathnbr             71987 non-null  float64
 5   calculatedfinishedsquarefeet  72055 non-null  float64
 6   finishedsquarefeet12          71875 non-null  float64
 7   fips                          72097 non-null  float64
 8   fullbathcnt                   71987 non-null  float64
 9   heatingorsystemtypeid         46697 non-null  float64
 10  latitude                      72097 non-null  float64
 11  longitude                     72097 non-null  float64
 12  lotsizesquarefeet             64007 non-null  float64
 13  p

In [13]:
# look at the following columns to fill values: buildingqualitytypeid, calculatedbathnbr, calculatedfinishedsquarefeet, finishedsquarefeet12, fullbathcnt, heatingorsystemtypeid, 
# replaced nulls with mode
df["buildingqualitytypeid"].fillna(8.00, inplace = True)

In [14]:
# replaced nulls with mode since we can't really have a fraction of a bathroom other than .5
df["calculatedbathnbr"].fillna(2.00, inplace = True)

In [15]:
# since the mode is 1200, the mean is 1750, and the 50% quartile is 1510, I'll go with 1510 to replace the few nulls we have
df["calculatedfinishedsquarefeet"].fillna(1510.00, inplace = True)


In [16]:
# replaced with 50% quartile 1510
df["finishedsquarefeet12"].fillna(1510.00, inplace = True)

In [17]:
# replacing with mode
df["fullbathcnt"].fillna(2.00, inplace = True)


In [18]:
# replaced with mode of 2 since that type of systemid is most common. 
df["heatingorsystemtypeid"].fillna(2.00, inplace = True)


In [19]:
# look at the following columns to fill values: lotsizesquarefeet, propertyzoningdesc, regionidcity, regionidzip, unitcnt, yearbuilt,structuretaxvaluedollarcnt, 
#replaced with mode
df["lotsizesquarefeet"].fillna(6000.00, inplace = True)

In [20]:
# dropped because it was missing too many nulls that should be unique. 
df = df.drop(columns='propertyzoningdesc')
df.head()

,parcelid,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,fips,fullbathcnt,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,propertycountylandusecode,propertylandusetypeid,rawcensustractandblock,regionidcity,regionidcounty,regionidzip,roomcnt,unitcnt,yearbuilt,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock,id.1,logerror,transactiondate
0,14297519,3.50,4.00,8.00,3.50,"3,100.00","3,100.00","6,059.00",3.00,2.00,"33,634,931.00","-117,869,207.00","4,506.00",122,261.00,"60,590,630.07","53,571.00","1,286.00","96,978.00",0.00,NaN,"1,998.00","485,713.00","1,023,282.00","2,016.00","537,569.00","11,013.72","60,590,630,072,012.00",0,0.03,2017-01-01
1,17052889,1.00,2.00,8.00,1.00,"1,465.00","1,465.00","6,111.00",1.00,2.00,"34,449,266.00","-119,281,531.00","12,647.00",1110,261.00,"61,110,010.02","13,091.00","2,061.00","97,099.00",5.00,NaN,"1,967.00","88,000.00","464,000.00","2,016.00","376,000.00","5,672.48","61,110,010,023,006.00",1,0.06,2017-01-01
2,14186244,2.00,3.00,8.00,2.00,"1,243.00","1,243.00","6,059.00",2.00,2.00,"33,886,168.00","-117,823,170.00","8,432.00",122,261.00,"60,590,218.02","21,412.00","1,286.00","97,078.00",6.00,NaN,"1,962.00","85,289.00","564,778.00","2,016.00","479,489.00","6,488.30","60,590,218,022,012.00",2,0.01,2017-01-01
3,12177905,3.00,4.00,8.00,3.00,"2,376.00","2,376.00","6,037.00",3.00,2.00,"34,245,180.00","-118,240,722.00","13,038.00",0101,261.00,"60,373,001.00","396,551.00","3,101.00","96,330.00",0.00,1.00,"1,970.00","108,918.00","145,143.00","2,016.00","36,225.00","1,777.51","60,373,001,001,006.00",3,-0.10,2017-01-01
4,10887214,3.00,3.00,8.00,3.00,"1,312.00","1,312.00","6,037.00",3.00,2.00,"34,185,120.00","-118,414,640.00","278,581.00",010C,266.00,"60,371,236.01","12,447.00","3,101.00","96,451.00",0.00,1.00,"1,964.00","73,681.00","119,407.00","2,016.00","45,726.00","1,533.89","60,371,236,012,000.00",4,0.01,2017-01-01


In [21]:
# drop rows with null for regionidcity, regionidzip, yearbuilt, structuretaxvaluedollarcnt since its under 2000 values missing. (will do last)
# both mean and mode for unitcnt are 1 so we'll replace nulls with 1
df["unitcnt"].fillna(1.00, inplace = True)

In [22]:
# drops rows with nulls since so few are missing (less than 100) structuretaxvaluedollarcnt
df.structuretaxvaluedollarcnt.describe(), df.structuretaxvaluedollarcnt.mode()

(count              72,016.00
 mean              188,131.09
 std               231,894.20
 min                    44.00
 25%                83,328.00
 50%               134,870.50
 75%               216,242.75
 max            11,421,790.00
 Name: structuretaxvaluedollarcnt, dtype: float64,
 0             100,000.00
 dtype: float64)

In [23]:
# look at the following columns to fill values: taxvaluedollarcnt, landtaxvaluedollarcnt, taxamount, censustractandblock
df.taxvaluedollarcnt.describe()
#drop the 1 row

count              72,096.00
mean              489,460.25
std               663,049.12
min                 1,000.00
25%               204,160.00
50%               356,000.00
75%               566,926.00
max            49,061,236.00
Name: taxvaluedollarcnt, dtype: float64

In [24]:
df.landtaxvaluedollarcnt.describe()
#drop the 1 row

count              72,096.00
mean              301,537.92
std               500,340.46
min                   161.00
25%                83,421.75
50%               203,141.00
75%               367,425.50
max            48,952,198.00
Name: landtaxvaluedollarcnt, dtype: float64

In [25]:
df.taxamount.describe()
#drop the few rows

count              72,092.00
mean                5,966.32
std                 7,730.31
min                    19.92
25%                 2,679.23
50%                 4,400.08
75%                 6,848.97
max               586,639.30
Name: taxamount, dtype: float64

In [26]:
df.censustractandblock.describe()
#drop rows with nulls and make object

count                71,882.00
mean     60,503,497,480,601.85
std       1,589,819,003,054.10
min      60,371,011,101,000.00
25%      60,374,002,053,566.75
50%      60,376,211,044,003.50
75%      60,590,524,173,014.00
max     483,030,105,084,015.00
Name: censustractandblock, dtype: float64

In [27]:
df.shape

(72097, 31)

In [28]:
df = df.dropna()
df.shape

(70441, 31)

In [29]:
df.head()

,parcelid,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,fips,fullbathcnt,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,propertycountylandusecode,propertylandusetypeid,rawcensustractandblock,regionidcity,regionidcounty,regionidzip,roomcnt,unitcnt,yearbuilt,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock,id.1,logerror,transactiondate
0,14297519,3.50,4.00,8.00,3.50,"3,100.00","3,100.00","6,059.00",3.00,2.00,"33,634,931.00","-117,869,207.00","4,506.00",122,261.00,"60,590,630.07","53,571.00","1,286.00","96,978.00",0.00,1.00,"1,998.00","485,713.00","1,023,282.00","2,016.00","537,569.00","11,013.72","60,590,630,072,012.00",0,0.03,2017-01-01
1,17052889,1.00,2.00,8.00,1.00,"1,465.00","1,465.00","6,111.00",1.00,2.00,"34,449,266.00","-119,281,531.00","12,647.00",1110,261.00,"61,110,010.02","13,091.00","2,061.00","97,099.00",5.00,1.00,"1,967.00","88,000.00","464,000.00","2,016.00","376,000.00","5,672.48","61,110,010,023,006.00",1,0.06,2017-01-01
2,14186244,2.00,3.00,8.00,2.00,"1,243.00","1,243.00","6,059.00",2.00,2.00,"33,886,168.00","-117,823,170.00","8,432.00",122,261.00,"60,590,218.02","21,412.00","1,286.00","97,078.00",6.00,1.00,"1,962.00","85,289.00","564,778.00","2,016.00","479,489.00","6,488.30","60,590,218,022,012.00",2,0.01,2017-01-01
3,12177905,3.00,4.00,8.00,3.00,"2,376.00","2,376.00","6,037.00",3.00,2.00,"34,245,180.00","-118,240,722.00","13,038.00",0101,261.00,"60,373,001.00","396,551.00","3,101.00","96,330.00",0.00,1.00,"1,970.00","108,918.00","145,143.00","2,016.00","36,225.00","1,777.51","60,373,001,001,006.00",3,-0.10,2017-01-01
4,10887214,3.00,3.00,8.00,3.00,"1,312.00","1,312.00","6,037.00",3.00,2.00,"34,185,120.00","-118,414,640.00","278,581.00",010C,266.00,"60,371,236.01","12,447.00","3,101.00","96,451.00",0.00,1.00,"1,964.00","73,681.00","119,407.00","2,016.00","45,726.00","1,533.89","60,371,236,012,000.00",4,0.01,2017-01-01


In [30]:
df.logerror

0                       0.03
1                       0.06
2                       0.01
3                      -0.10
4                       0.01
                ...         
72236                   0.02
72237                   0.01
72238                   0.04
72239                   0.01
72240                  -0.20
Name: logerror, Length: 70441, dtype: float64

In [31]:
df.head()

,parcelid,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,fips,fullbathcnt,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,propertycountylandusecode,propertylandusetypeid,rawcensustractandblock,regionidcity,regionidcounty,regionidzip,roomcnt,unitcnt,yearbuilt,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock,id.1,logerror,transactiondate
0,14297519,3.50,4.00,8.00,3.50,"3,100.00","3,100.00","6,059.00",3.00,2.00,"33,634,931.00","-117,869,207.00","4,506.00",122,261.00,"60,590,630.07","53,571.00","1,286.00","96,978.00",0.00,1.00,"1,998.00","485,713.00","1,023,282.00","2,016.00","537,569.00","11,013.72","60,590,630,072,012.00",0,0.03,2017-01-01
1,17052889,1.00,2.00,8.00,1.00,"1,465.00","1,465.00","6,111.00",1.00,2.00,"34,449,266.00","-119,281,531.00","12,647.00",1110,261.00,"61,110,010.02","13,091.00","2,061.00","97,099.00",5.00,1.00,"1,967.00","88,000.00","464,000.00","2,016.00","376,000.00","5,672.48","61,110,010,023,006.00",1,0.06,2017-01-01
2,14186244,2.00,3.00,8.00,2.00,"1,243.00","1,243.00","6,059.00",2.00,2.00,"33,886,168.00","-117,823,170.00","8,432.00",122,261.00,"60,590,218.02","21,412.00","1,286.00","97,078.00",6.00,1.00,"1,962.00","85,289.00","564,778.00","2,016.00","479,489.00","6,488.30","60,590,218,022,012.00",2,0.01,2017-01-01
3,12177905,3.00,4.00,8.00,3.00,"2,376.00","2,376.00","6,037.00",3.00,2.00,"34,245,180.00","-118,240,722.00","13,038.00",0101,261.00,"60,373,001.00","396,551.00","3,101.00","96,330.00",0.00,1.00,"1,970.00","108,918.00","145,143.00","2,016.00","36,225.00","1,777.51","60,373,001,001,006.00",3,-0.10,2017-01-01
4,10887214,3.00,3.00,8.00,3.00,"1,312.00","1,312.00","6,037.00",3.00,2.00,"34,185,120.00","-118,414,640.00","278,581.00",010C,266.00,"60,371,236.01","12,447.00","3,101.00","96,451.00",0.00,1.00,"1,964.00","73,681.00","119,407.00","2,016.00","45,726.00","1,533.89","60,371,236,012,000.00",4,0.01,2017-01-01


In [32]:
# look for duplicates
df.parcelid.duplicated().sum()

165

In [33]:
df.shape

(70441, 31)

In [34]:
df = df.drop_duplicates(subset='parcelid')

In [35]:
df.shape

(70276, 31)

In [36]:
df.head()

,parcelid,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,fips,fullbathcnt,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,propertycountylandusecode,propertylandusetypeid,rawcensustractandblock,regionidcity,regionidcounty,regionidzip,roomcnt,unitcnt,yearbuilt,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock,id.1,logerror,transactiondate
0,14297519,3.50,4.00,8.00,3.50,"3,100.00","3,100.00","6,059.00",3.00,2.00,"33,634,931.00","-117,869,207.00","4,506.00",122,261.00,"60,590,630.07","53,571.00","1,286.00","96,978.00",0.00,1.00,"1,998.00","485,713.00","1,023,282.00","2,016.00","537,569.00","11,013.72","60,590,630,072,012.00",0,0.03,2017-01-01
1,17052889,1.00,2.00,8.00,1.00,"1,465.00","1,465.00","6,111.00",1.00,2.00,"34,449,266.00","-119,281,531.00","12,647.00",1110,261.00,"61,110,010.02","13,091.00","2,061.00","97,099.00",5.00,1.00,"1,967.00","88,000.00","464,000.00","2,016.00","376,000.00","5,672.48","61,110,010,023,006.00",1,0.06,2017-01-01
2,14186244,2.00,3.00,8.00,2.00,"1,243.00","1,243.00","6,059.00",2.00,2.00,"33,886,168.00","-117,823,170.00","8,432.00",122,261.00,"60,590,218.02","21,412.00","1,286.00","97,078.00",6.00,1.00,"1,962.00","85,289.00","564,778.00","2,016.00","479,489.00","6,488.30","60,590,218,022,012.00",2,0.01,2017-01-01
3,12177905,3.00,4.00,8.00,3.00,"2,376.00","2,376.00","6,037.00",3.00,2.00,"34,245,180.00","-118,240,722.00","13,038.00",0101,261.00,"60,373,001.00","396,551.00","3,101.00","96,330.00",0.00,1.00,"1,970.00","108,918.00","145,143.00","2,016.00","36,225.00","1,777.51","60,373,001,001,006.00",3,-0.10,2017-01-01
4,10887214,3.00,3.00,8.00,3.00,"1,312.00","1,312.00","6,037.00",3.00,2.00,"34,185,120.00","-118,414,640.00","278,581.00",010C,266.00,"60,371,236.01","12,447.00","3,101.00","96,451.00",0.00,1.00,"1,964.00","73,681.00","119,407.00","2,016.00","45,726.00","1,533.89","60,371,236,012,000.00",4,0.01,2017-01-01


In [37]:
def remove_outliers(df, k, col_list):
    ''' remove outliers from a list of columns in a dataframe 
        and return that dataframe
    '''
    
    for col in col_list:

        q1, q3 = df[col].quantile([.25, .75])  # get quartiles
        
        iqr = q3 - q1   # calculate interquartile range
        
        upper_bound = q3 + k * iqr   # get upper bound
        lower_bound = q1 - k * iqr   # get lower bound

        # return dataframe without outliers
        
        df = df[(df[col] > lower_bound) & (df[col] < upper_bound)]
        
    return df

In [38]:
df['area'] = df.calculatedfinishedsquarefeet
df['bed'] = df.bedroomcnt
df['bath'] = df.bathroomcnt
df['taxvalue'] = df.taxvaluedollarcnt
df = df.drop(columns=['calculatedfinishedsquarefeet','bedroomcnt', 'bathroomcnt', 'taxvaluedollarcnt'])
df = df.dropna()
col_list = ['bed', 'bath', 'area', 'taxvalue', 'taxamount']
df = remove_outliers(df, 1, col_list)
df.shape
# assuming the list of variables from the previous project hold value in predicting logerror. 

(44618, 31)

In [39]:
df.head()

,parcelid,buildingqualitytypeid,calculatedbathnbr,finishedsquarefeet12,fips,fullbathcnt,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,propertycountylandusecode,propertylandusetypeid,rawcensustractandblock,regionidcity,regionidcounty,regionidzip,roomcnt,unitcnt,yearbuilt,structuretaxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock,id.1,logerror,transactiondate,area,bed,bath,taxvalue
2,14186244,8.00,2.00,"1,243.00","6,059.00",2.00,2.00,"33,886,168.00","-117,823,170.00","8,432.00",122,261.00,"60,590,218.02","21,412.00","1,286.00","97,078.00",6.00,1.00,"1,962.00","85,289.00","2,016.00","479,489.00","6,488.30","60,590,218,022,012.00",2,0.01,2017-01-01,"1,243.00",3.00,2.00,"564,778.00"
3,12177905,8.00,3.00,"2,376.00","6,037.00",3.00,2.00,"34,245,180.00","-118,240,722.00","13,038.00",0101,261.00,"60,373,001.00","396,551.00","3,101.00","96,330.00",0.00,1.00,"1,970.00","108,918.00","2,016.00","36,225.00","1,777.51","60,373,001,001,006.00",3,-0.10,2017-01-01,"2,376.00",4.00,3.00,"145,143.00"
4,10887214,8.00,3.00,"1,312.00","6,037.00",3.00,2.00,"34,185,120.00","-118,414,640.00","278,581.00",010C,266.00,"60,371,236.01","12,447.00","3,101.00","96,451.00",0.00,1.00,"1,964.00","73,681.00","2,016.00","45,726.00","1,533.89","60,371,236,012,000.00",4,0.01,2017-01-01,"1,312.00",3.00,3.00,"119,407.00"
5,17143294,8.00,2.00,"1,492.00","6,111.00",2.00,2.00,"34,230,044.00","-118,993,991.00",903.00,1129,266.00,"61,110,053.06","51,239.00","2,061.00","97,091.00",6.00,1.00,"1,982.00","198,640.00","2,016.00","132,424.00","3,508.10","61,110,053,062,009.00",5,-0.02,2017-01-01,"1,492.00",3.00,2.00,"331,064.00"
9,11542646,8.00,3.00,"1,290.00","6,037.00",3.00,2.00,"33,998,800.00","-118,416,000.00","54,048.00",010C,266.00,"60,372,751.02","12,447.00","3,101.00","96,047.00",0.00,1.00,"1,980.00","151,303.00","2,016.00","220,058.00","4,557.32","60,372,751,022,006.00",9,-0.04,2017-01-02,"1,290.00",2.00,3.00,"371,361.00"
